In [15]:
from gensim.models import FastText
import os,sys
from urllib.parse import urlparse
from getImagesFunctions import *
from getDataFunctions import *
from gensim.models import Word2Vec
import gensim
import random
import langid
import re
import ast
import concurrent
import htmldate
import time
import uuid

In [13]:
base_path = os.getcwd()
name_folder = "image_npg_"

In [14]:
with open('C:/Users/Ruben/Documents/GitHub/ReACT_GCV/code/work/scrape_environment/npg_source/page-urls.txt') as f:
    urls= json.load(f)
    urls = dict(urls)
    
for k,v in urls.items():
    print(k,len(v))

1 256
2 26333
3 74345
4 77331
5 86160


In [10]:
MAX_THREADS = 30

def download_url(url):
    title = str(uuid.uuid4()) + '.txt'
    d = find_date(url)
    print(d,title)
    with open(title, "w") as fh:
        f.write("{}|{}".format(url,d))
        
    time.sleep(0.25)
    
def download_all(list_urls):
    threads = min(MAX_THREADS, len(list_urls))
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
        executor.map(download_url, list_urls)

def main(list_urls):
    t0 = time.time()
    download_all(list_urls)
    t1 = time.time()
    print(f"{t1-t0} seconds to download {len(list_urls)} dates.")

In [ ]:
test_path = os.path.join(base_path, "test")
if not os.path.exists(test_path):
    os.makedirs(test_path)
os.chdir(test_path)
main(urls['1'])

In [11]:
download_url(urls['1'][0])

2019-12-05 httpswwwinquirercomartsgirlinvietnampicturekimphuchanniballocumbehtml.txt


TypeError: a bytes-like object is required, not 'str'

In [109]:
urlsp = [[x for x in urlparse(i) if i][2:] for i in urls]
urlsp = [[x for x in i if x] for i in urlsp]
urlsp = [" ".join(x) for x in urlsp]

In [111]:
def SearchURLMatch(url):
    poss_years = [str(i) for i in range(1990,2021)]
    poss_months_char = {"jan":'01', "feb":'02', "mar":'03', "apr":'04', "may":'05', "jun":'06', "jul":'07', "aug":'08',"sep":'09', "oct":'10', "nov":'11', "dec":'12'}
    poss_months_int = "01 02 03 04 05 06 07 08 09 10 11 12"
    poss_days_int = [str(i) for i in range(1,32)]
    poss_days_int = ["0"+i for i in poss_days_int if len(i) == 1] + [i for i in poss_days_int if len(i) > 1]
    
    u = url.split('/')
    doubts = "no"
    year = "na"
    month = 'na'
    day = 'na'
    
    if any(y in u for y in poss_years):
        index_year = u.index([y for y in u if y in poss_years][0])
        
        # IF MONTH IS A STRING: "JAN" OR "OCT"
        if u[index_year+1] in poss_months_char.keys():
            year = u[index_year]
            month = poss_months_char[u[index_year+1]]
            if u[index_year+2] in poss_days_int:
                day = u[index_year+2]
                status = "found something"
            if u[index_year+2] not in poss_days_int:
                day = "na"
            
            
        # IF PATTERN IS YEAR-MONTH-DAY
        try:
            if u[index_year+1] in poss_months_int and u[index_year+2] in poss_days_int:
                year = u[index_year]
                month = u[index_year+1]
                day = u[index_year+2]
                status = "found something"
                if u[index_year+2] in poss_months_int and u[index_year+1] in poss_days_int and u[index_year+1] != u[index_year+2]:
                    doubts = "yes"
        except IndexError:
            doubts = "yes"

        
        # IF PATTERN IS YEAR-DAY-MONTH
        try:
            if u[index_year+1] in poss_days_int and u[index_year+2] in poss_months_int:
                year = u[index_year]
                month = u[index_year+2]
                day = u[index_year+1]
                status = "found something"
                if u[index_year+1] in poss_months_int and u[index_year+2] in poss_days_int and u[index_year+2] != u[index_year+1]:
                        doubts = "yes"
        except IndexError:
                doubts = "yes"
        
        # IF PATTERN IS MONTH-DAY-YEAR
        try:
            if u[index_year-1] in poss_days_int and u[index_year-2] in poss_months_int:
                year = u[index_year]
                month = u[index_year-2]
                day = u[index_year-1]
                status = "found something"
                if u[index_year-1] in poss_months_int and u[index_year-2] in poss_days_int and u[index_year-1] != u[index_year-2]:
                    doubts = "yes"
        except IndexError:
                doubts = "yes"
        
        # IF PATTERN IS DAY-MONTH-YEAR
        try:
            if u[index_year-2] in poss_days_int and u[index_year-1] in poss_months_int:
                year = u[index_year]
                month = u[index_year-1]
                day = u[index_year-2]
                status = "found something"
                if u[index_year-2] in poss_months_int and u[index_year-1] in poss_days_int and u[index_year-2] != u[index_year-1]:
                    doubts = "yes"
        except IndexError:
                doubts = "yes"
       
        status = "found something"
        
        if doubts == "no" and status == "found something" and year != "na" and month != "na" and day != "na":
            return [year,month,day]
        
        elif doubts == "yes":
            return ["na"]
    else:
        status = "found nothing"


In [112]:
total = 0
result = 0

d_ = dict()

for u in urlsp:
    
    res = SearchURL(u)
    
    if res and res != ['na']:
        #print(u)
        #print(res)
        #print('--------------------')
        d_.update({u:res})
        result += 1
        total += 1
    else:
        total += 1
        
        

In [114]:
total

77331